In [ ]:
#| default_exp specials
from nbdev import *
from nbdev.showdoc import *
import networkx as nx

## Specific algo's

In [ ]:
#| export
import hashlib
from pathlib import Path
import networkx as nx

In [ ]:
#| export
def md5(input):
    return hashlib.md5(input.encode('utf-8')).hexdigest()


In [ ]:
md5('bla')

'128ecf542a35ac5270a87dc740918404'

In [ ]:
#| export

def binarysearch(minim,maxim,function, flips_to_true=True): 
    """
     function needs to return a boolean whether the solution is ok
     this implementation is for function that starts with false for minim and flip to true
     for TTTTFFFF, pass set flips_to_true flag to false. This flag is important to set correct!
    """
    new = minim
    while True:
        new = (minim+maxim)//2
        print(f'to_test: {new}, min {minim}, max {maxim} ', end=' ')
        res = function(new)
        print('function returns', res)
        if not flips_to_true: res = not res
        if res:
            if new == maxim: # solution found
                if flips_to_true:
                    print('solution found',new)
                    return new
                else:
                    print('solution found',new-1)
                    return new-1
            maxim = new
        else: minim = new+1


In [ ]:
assert binarysearch(0,200, lambda x: x > 50) == 51

to_test: 100, min 0, max 200  function returns True
to_test: 50, min 0, max 100  function returns False
to_test: 75, min 51, max 100  function returns True
to_test: 63, min 51, max 75  function returns True
to_test: 57, min 51, max 63  function returns True
to_test: 54, min 51, max 57  function returns True
to_test: 52, min 51, max 54  function returns True
to_test: 51, min 51, max 52  function returns True
to_test: 51, min 51, max 51  function returns True
solution found 51


In [ ]:
assert binarysearch(0,200, lambda x: x < 50, flips_to_true=False) == 49

to_test: 100, min 0, max 200  function returns False
to_test: 50, min 0, max 100  function returns False
to_test: 25, min 0, max 50  function returns True
to_test: 38, min 26, max 50  function returns True
to_test: 44, min 39, max 50  function returns True
to_test: 47, min 45, max 50  function returns True
to_test: 49, min 48, max 50  function returns True
to_test: 50, min 50, max 50  function returns False
solution found 49


In [ ]:
#| export
def deduce_matches(input_dict, option_type=str):
    """
    Takes a dict with multiple keys that have one or more options
    The trick is to start with what you know: keys with one option and remove that option for the other keys
    Continuing that process leads to every key ending up with one option (hopefully)

    Assumes: the options are strings and sort in some kind of container
    """
    found = 0
    while found < len(input_dict):
        for k, v in input_dict.items():
            if not isinstance(v, option_type) and len(v) == 1: # found one
                to_rem = v.pop()
                input_dict[k] = to_rem
                found += 1
                for _ , v2 in input_dict.items(): # remove the item from other lists
                    if not isinstance(v2, option_type) and to_rem in v2:
                        v2.remove(to_rem)
    return input_dict

In [ ]:
#| export
def find_pattern_in_iter(start_pattern, function, goal = None, n_iter=1000000000):
    """
        Returns when a SPECIFIED pattern has been found from a function
        If goal = None, then first time the start pattern shows up again is returned
        Returns steps, pattern
    """
    if not goal: goal = start_pattern
    current = start_pattern
    for i in range(1,n_iter):
        current = function(current)
        # print(current)
        if current == goal:
            print(f'At step {i}, goal: {current} was found')
            return i, current

In [ ]:
#| export
def find_repeat(start_pattern, function, n_iter=None):
    """
        Returns when a NONSPECFIED repeating pattern has been found
        Returns steps, pattern
    """
    if not n_iter: n_iter = round(10e20)
    seen = {start_pattern}
    current = start_pattern
    for i in range(1,n_iter):
        current = function(current)
        # print(current)
        if current in seen:
            print(f'Repeat was found at step {i}. Pattern: {current}')
            return i,current
        seen.add(current)

In [ ]:
#| export
def find_cycle(start_pattern, function):
    """
        Find cycle length of some repeating pattern, by first inspecting which item repeats when
        And subtracting the time the item was first seen
    """
    step_second, pattern = find_repeat(start_pattern, function)
    step_first, pattern = find_pattern_in_iter(start_pattern, function, goal = pattern)
    return step_second - step_first

In [ ]:
#| hide
class Test_gen():
    def __init__(self):
        self.results = iter([5,10,15,5,99,10])
    def __call__(self,*args):
        return next(self.results)



In [ ]:
assert find_pattern_in_iter(99,Test_gen(),n_iter=10) == (5,99)
assert find_pattern_in_iter(99,Test_gen(),goal=10, n_iter=10) == (2,10)
assert find_pattern_in_iter(99,Test_gen(),goal=5, n_iter=10) == (1,5)
assert find_repeat(99,Test_gen(),n_iter=10) == (4,5)

found at step 5 99
found at step 2 10
found at step 1 5
found at step 4 5


In [ ]:
#| hide
# from nbdev.export import notebook2script; 
# notebook2script()
# !nbdev_build_lib
# !nbdev_build_docs
# !nbdev_clean_nbs
# !git add .
# !git commit -am "change future upwards"
# !git push

Converted 00_core.ipynb.
Converted 01_context_free_grammar.ipynb.
Converted 02_norvig.ipynb.
Converted index.ipynb.
Converted 00_core.ipynb.
Converted 01_context_free_grammar.ipynb.
Converted 02_norvig.ipynb.
Converted index.ipynb.
converting: d:\Documenten\GitHub\adventofcode\aocutils\00_core.ipynb
converting: d:\Documenten\GitHub\adventofcode\aocutils\01_context_free_grammar.ipynb
converting: d:\Documenten\GitHub\adventofcode\aocutils\02_norvig.ipynb
converting: d:\Documenten\GitHub\adventofcode\aocutils\index.ipynb
converting d:\Documenten\GitHub\adventofcode\aocutils\index.ipynb to README.md
[main 47c0ec4] change future upwards
 3 files changed, 14 insertions(+), 7 deletions(-)
To https://github.com/jvanelteren/aocutils.git
   68e7b8a..47c0ec4  main -> main
